In [1]:
import numpy as np
import pandas as pd
from glob import glob
import sys
import os
from datetime import datetime

import matplotlib.pyplot as plt

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import ccxt

sys.path.append("..") 
from tp_config import *
from tp_utils.data_provider import read_prices, read_data,load_data_from_exchange
from portfolio_tools import (load_data_for_portfolio, calc_frontier,
                             calc_weights, weights_to_df, print_data)

## Helper functions

In [52]:
now = datetime.today().strftime('%Y-%m-%d')
def start_end_win(period, offset, horizon, lookback):
    start_win = offset + period * horizon
    end_win = start_win + lookback
    return (start_win, end_win)


In [281]:
data_path = DATA_PATH_CRYPTO

pair = 'USDT'
#pair = 'BTC'

index_assets = pd.read_csv('index_assets.csv')['asset'].tolist()
index_low_assets = pd.read_csv('index_low_assets.csv')['asset'].tolist()
assets = index_assets + index_low_assets
markets = [m + '-' + pair for m in assets]

### Data from exchange

In [303]:
exchange = ccxt.bybit()
start_timestamp = exchange.parse8601('2024-01-01 00:00:00')
timeframes = {'1h':60, '4h':240, '12h':720}

data_path = DATA_PATH_CRYPTO + '/' + pair
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, 10, verbose = False)

Done
Time taken = 0.017  hours


## Read from local data

In [304]:
df_all =  load_data_for_portfolio(markets, '4h')   
print(datetime.fromtimestamp(df_all.index[0]/1000))
print(datetime.fromtimestamp(df_all.index[-1]/1000))

AVAX-USDT
BCH-USDT
BTC-USDT
BNB-USDT
DOT-USDT
DOGE-USDT
ETH-USDT
LINK-USDT
LTC-USDT
LUNA-USDT
MATIC-USDT
MNT-USDT
SOL-USDT
TON-USDT
TRX-USDT
AAVE-USDT
ADA-USDT
AEVO-USDT
ALGO-USDT
APT-USDT
AXS-USDT
BONK-USDT
BOME-USDT
CAKE-USDT
CORE-USDT
COQ-USDT
CYBER-USDT
CPOOL-USDT
EOS-USDT
ETC-USDT
ETHFI-USDT
FIRE-USDT
FLT-USDT
FTT-USDT
FTM-USDT
GALA-USDT
INJ-USDT
ICP-USDT
JUP-USDT
MASK-USDT
NEON-USDT
NEAR-USDT
ONDO-USDT
PEPE-USDT
PENDLE-USDT
PPT-USDT
PYTH-USDT
RUNE-USDT
RVN-USDT
SEI-USDT
SQT-USDT
SUI-USDT
THETA-USDT
TOKEN-USDT
UNI-USDT
WAVES-USDT
WLD-USDT
XRP-USDT
ZIL-USDT
2024-01-01 04:00:00
2024-04-01 08:00:00


In [305]:
since = 1703874400000
df_prices = df_all[df_all.index > since]

print(datetime.fromtimestamp(df_prices.index[0]/1000))
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

2024-01-01 04:00:00
2024-04-01 08:00:00


In [301]:
import riskfolio as rp


def riskfolio_weights(df_period, obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    rm = 'CVaR' # Risk measure used, this time will be variance
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 2
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

In [235]:

def index_date(ind):
    return datetime.fromtimestamp(ind/1000)

class Portfolio:
    def __init__(self, balance):
        self.balance   = balance
        self.portfolio = {}
        self.buy_correction = 1.001
        self.sell_correction = 0.99
        
    def sell_portfolio(self, prices):
        for key in self.portfolio.keys():
            asset_price = prices[key]
            self.balance = self.balance + self.portfolio[key] * asset_price * self.sell_correction
        self.portfolio = {}
        
    def buy_portfolio(self, portfolio,  prices):
        overall_balance = self.balance
        for key in portfolio.keys():
            asset_price = prices[key]
            sum_for_asset = portfolio[key] * overall_balance 
            self.balance = self.balance - sum_for_asset 
            quant = sum_for_asset / (asset_price * 1.00075)
            self.portfolio[key] = quant

In [289]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

return_methods = [
    "mean_historical_return",
    "ema_historical_return",
    "capm_return", 
    ]

In [309]:
lookback = 120
rebalance_period = 12
offset = 7
start_trade = lookback + offset

port = Portfolio(100)

count = 0 

for ind, row in df_prices.iloc[start_trade:].iterrows():
    if count % rebalance_period == 0:
        print('Rebalance data: ', ind, index_date(ind))
        
        port.sell_portfolio(row) 
        print("Баланс портфеля", port.balance)
        
        df_period = df_prices.loc[:ind]
        df_period = df_period[-lookback:]
        df_period = df_period.dropna(axis = 1)
        
#        ef  = calc_frontier(df_period, "semicovariance",  "ema_historical_return", span = 150)
        ef  = calc_frontier(df_period, "ledoit_wolf",  "ema_historical_return", span = 150)
        dfw = calc_weights(ef, 'max_sharpe', 0)      
        
#        dfw = riskfolio_weights(df_period, "MaxRet")
        
        new_portfolio = dfw.to_dict()['weights']
        print("Новый портфель: ", new_portfolio)
        
        port.buy_portfolio(new_portfolio, row)
         
        print()
    count = count + 1

Rebalance data:  1705896000000 2024-01-22 08:00:00
Баланс портфеля 100
Новый портфель:  {'PENDLE-USDT': 0.7658900000000001, 'PPT-USDT': 0.13535000000000003, 'PYTH-USDT': 0.09876000000000001}

Rebalance data:  1706068800000 2024-01-24 08:00:00
Баланс портфеля 89.7961247450056
Новый портфель:  {'PENDLE-USDT': 0.46838468384683846, 'PYTH-USDT': 0.1083310833108331, 'SUI-USDT': 0.4232842328423284}

Rebalance data:  1706241600000 2024-01-26 08:00:00
Баланс портфеля 95.58582440991306
Новый портфель:  {'TRX-USDT': 0.05784, 'PENDLE-USDT': 0.68052, 'PYTH-USDT': 0.26164}

Rebalance data:  1706414400000 2024-01-28 08:00:00
Баланс портфеля 90.09493093757942
Новый портфель:  {'PENDLE-USDT': 0.23119, 'PYTH-USDT': 0.23622, 'SUI-USDT': 0.53259}

Rebalance data:  1706587200000 2024-01-30 08:00:00
Баланс портфеля 100.2324665913517
Новый портфель:  {'PENDLE-USDT': 0.34839, 'SUI-USDT': 0.65161}

Rebalance data:  1706760000000 2024-02-01 08:00:00
Баланс портфеля 93.10295605741578
Новый портфель:  {'PENDLE-US

## Mass cov

In [252]:
%%time
res = []

offset = 8
start_trade = lookback + offset

for lookback in range(70, 120):
    print(" ", lookback)
    for rebalance_period in range(9, 20):
         
        port  = Portfolio(100)
        count = 0
        for ind, row in df_prices.iloc[start_trade:].iterrows():
            if count % rebalance_period == 0:
#                print('Rebalance data: ', ind, index_date(ind))

                port.sell_portfolio(row) 
#                print("Баланс портфеля", port.balance)

                df_period = df_prices.loc[:ind]
                df_period = df_period[-lookback:]
                df_period = df_period.dropna(axis = 1)

        #        ef  = calc_frontier(df_period, "semicovariance",  "ema_historical_return", span = 150)
        #        ef  = calc_frontier(df_period, "ledoit_wolf",  "ema_historical_return", span = 150)
        #        dfw = calc_weights(ef, 'max_sharpe', 0)      

                dfw = riskfolio_weights(df_period, "MaxRet")

                new_portfolio = dfw.to_dict()['weights']
  #              print("Новый портфель: ", new_portfolio)

                port.buy_portfolio(new_portfolio, row)

 #               print()
            count = count + 1
    
        port.sell_portfolio(row) 
        print("   ", rebalance_period, port.balance)
        res.append([lookback, rebalance_period, port.balance])   

  60
    9 182.96908703504272
    10 178.63147344902183
    11 179.59735417523194
    12 189.70093688416995
    13 195.91975738790944
    14 207.9070507873153
You must convert self.cov to a positive definite matrix
    15 210.1291810672099
You must convert self.cov to a positive definite matrix
    16 190.19127731428853
    17 216.08202619617924
    18 237.0611372799559
    19 184.1235975402323
  61
    9 169.19102197784673
    10 166.93374750262043
    11 160.77414278873377
    12 191.79511309644312
    13 198.4711518430534
    14 200.64480589381182
    15 211.706063824002
    16 168.48862937903414
    17 202.23786808754699
    18 228.64805519409606
    19 187.1787836074579
  62
    9 164.23843335377035
    10 164.64688541397356
    11 166.61166111369786
    12 178.55687494545202
    13 190.67939286695594
    14 197.84910837987763
    15 205.98767862088778
    16 157.8618010151576
    17 201.71627994751807
    18 219.89652048071684
    19 170.5796164054335
  63
    9 169.49441740646
 

In [246]:
df = pd.DataFrame(res, columns = ['look', 'horizon', 'balance'])
dfx = df.sort_values('balance', ascending = False)
 

In [247]:
dfx.groupby(['look']).mean().sort_values('balance')

,horizon,balance
look,,
60,8.5,NaN
61,8.5,NaN
62,8.5,NaN
63,8.5,NaN
64,8.5,NaN
65,8.5,NaN
66,8.5,NaN
67,8.5,NaN
68,8.5,NaN


### Mass offset

In [284]:
res = []
offset = 0

for method in risk_methods: 
    print(method)
    for lookback in range(89, 92):
        print(" ", lookback)
        for horizon in [5,6,8, 9]:
            for offset in range(0,24):
                n_periods = (df_prices.shape[0]  - lookback - offset) // horizon - 1
                balance = 0.10
                portfolio = {}
                for period in range(n_periods):

                    start_win, end_win = start_end_win(period, offset, horizon, lookback)

                # Sell portfolio    

            #        print(balance)    
                    df_period = df_prices.iloc[start_win:end_win]
                    ef = calc_frontier(df_period, method)
                    try:
                        dfw = calc_weights(ef, 'max_sharpe', 0) 
                    except:
                        continue

                #Buy portfolio    
                    portfolio = dfw.to_dict()['W']
    #                print(portfolio)
                    portfolio, balance = buy_portfolio(end_win, balance)

                # Sell portfolio      
                    balance = sell_portfolio(portfolio, end_win + horizon, balance)
                    cur_time = datetime.fromtimestamp(df_prices.index[end_win+horizon]/1000)
    #                print(period, cur_time , balance)    


                    res.append([method, lookback, horizon, offset, n_periods, balance])        

semicovariance
  89
  90
  91
ledoit_wolf_constant_correlation
  89
  90
  91


In [292]:
df = pd.DataFrame(res, columns = ['method', 'look', 'horizon', 'offset','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx.groupby(['method','look', 'horizon' ]).mean().sort_values('balance')

offset             n  \
method                           look horizon                               
ledoit_wolf_constant_correlation 91   5        11.233065442  36.347876005   
                                 90   5        11.231735160  36.557077626   
                                      6        11.229281768  30.215469613   
                                      9        11.216560510  19.662420382   
                                 89   5        11.236095346  36.763904654   
                                 91   9        11.211538462  19.538461538   
                                      6        11.225000000  30.050000000   
                                      8        11.239171375  22.160075330   
                                 89   6        11.239010989  30.379120879   
                                 90   8        11.226591760  22.288389513   
                                 89   9        11.227848101  19.784810127   
semicovariance                   91   5        11.233065442  36.347876005   
ledoit_wolf_constant_correlation 89   8        11.219739292  22.415270019   
semicovariance                   91   6        11.225000000  30.050000000   
                                      9        11.211538462  19.538461538   
                                      8        11.239171375  22.160075330   
                                 90   6        11.229281768  30.215469613   
                                      9        11.216560510  19.662420382   
                                      5        11.231735160  36.557077626   
                                 89   5        11.236095346  36.763904654   
                                 90   8        11.226591760  22.288389513   
                                 89   9        11.227848101  19.784810127   
                                      6        11.239010989  30.379120879   
                                      8        11.219739292  22.415270019   

                                                   balance  
method                           look horizon               
ledoit_wolf_constant_correlation 91   5        0.127415394  
                                 90   5        0.127716651  
                                      6        0.127924961  
                                      9        0.127941528  
                                 89   5        0.127996540  
                                 91   9        0.128014798  
                                      6        0.128110338  
                                      8        0.128145928  
                                 89   6        0.128309196  
                                 90   8        0.128335053  
                                 89   9        0.128353634  
semicovariance                   91   5        0.128397841  
ledoit_wolf_constant_correlation 89   8        0.128462395  
semicovariance                   91   6        0.129114217  
                                      9        0.129166876  
                                      8        0.129203905  
                                 90   6        0.129546970  
                                      9        0.129672050  
                                      5        0.129681249  
                                 89   5        0.129838040  
                                 90   8        0.130105575  
                                 89   9        0.130506244  
                                      6        0.130511691  
                                      8        0.130532659

(291, 29)

In [274]:
df = pd.DataFrame(res, columns = ['method', 'look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[(dfx['look'] > 85) & (dfx['look'] < 92)]
dfx[:20]

,method,look,horizon,n,balance
1969,semicovariance,89,8,25,0.197796092
2139,semicovariance,90,9,22,0.193421789
2031,semicovariance,90,5,40,0.191268967
1767,semicovariance,88,6,34,0.190027756
2179,semicovariance,91,5,40,0.189497499
3578,semicovariance,100,9,21,0.188080246
3754,semicovariance,102,5,38,0.187885951
3755,semicovariance,102,5,38,0.187838461
1991,semicovariance,89,9,22,0.187554015
2387,semicovariance,92,7,28,0.187062521


In [228]:
dfx.groupby(['horizon']).mean().sort_values('balance')

,look,n,balance
horizon,,,
9,90.0,14.08,0.139155017
8,90.0,16.08,0.139321131
6,90.0,21.92,0.142085156
7,90.0,18.56,0.142215123
5,90.0,26.60,0.143235112


In [126]:
### Mass

In [ ]:
res = []
offset = 0

for lookback in range(72, 102):
    print(lookback)
    for horizon in range(5,11):

        n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
        balance = 0.10
        portfolio = {}
        all_balance = 0
        for period in range(n_periods):

            start_win, end_win = start_end_win(period, horizon, lookback)

        # Sell portfolio    

    #        print(balance)    
            df_period = df_prices.iloc[start_win:end_win]
            ef = calc_frontier(df_period, 'ledoit_wolf')
            try:
                dfw = calc_weights(ef, 'max_sharpe', 0) 
            except:
                continue

            balance = sell_portfolio(portfolio, end_win, balance)
            all_balance = balance
        #Buy portfolio    
            portfolio = dfw.to_dict()['W']
            portfolio, balance = buy_portfolio(end_win, balance)

        res.append([lookback, horizon, n_periods, all_balance])

In [ ]:
df = pd.DataFrame(res, columns = ['look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

In [230]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

In [ ]:
res = []
offset = 0
#0.35, 95, 10
for par in [0.35, 0.4, 0.45, 0.5]:
    for cov_method in risk_methods:
        for lookback in range(80, 100):
            print(lookback)
            for horizon in range(2,12):
                profit = 0
                n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
                balance = 0.1
                portfolio = {}
                all_balance = 0
                for period in range(n_periods):

                    start_win, end_win = start_end_win(period, horizon, lookback)

                # Sell portfolio    

            #        print(balance)    
                    df_period = df_prices.iloc[start_win:end_win]
                    ef = calc_frontier(df_period, cov_method)
                    dfw = calc_weights(ef, 'efficient_risk', par) 
                    try:
                        pass
                    except:
                        continue

                    balance = sell_portfolio(portfolio, end_win, balance)
                    all_balance = balance
                    profit = profit + balance - 0.1
                #Buy portfolio    
                    balance = 0.10
                    portfolio = dfw.to_dict()['W']
                    portfolio, balance = buy_portfolio(end_win, balance)

                res.append([par, cov_method, lookback, horizon, n_periods, all_balance, profit])

In [232]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20] 

,par,cov,look,horizon,n,balance,profit
2625,0.40,ledoit_wolf_single_factor,82,7,20,0.107254666,0.076422891
2825,0.40,ledoit_wolf_constant_correlation,82,7,20,0.107145394,0.076284547
2904,0.40,ledoit_wolf_constant_correlation,90,6,22,0.107917221,0.075631286
2104,0.40,exp_cov,90,6,22,0.107943484,0.075624305
1625,0.40,sample_cov,82,7,20,0.107233674,0.075388251
2704,0.40,ledoit_wolf_single_factor,90,6,22,0.107904749,0.075289034
2813,0.40,ledoit_wolf_constant_correlation,81,5,28,0.103346873,0.075159280
1704,0.40,sample_cov,90,6,22,0.107906898,0.074867175
2877,0.40,ledoit_wolf_constant_correlation,87,9,14,0.132459390,0.074714529
4425,0.45,ledoit_wolf_constant_correlation,82,7,20,0.108042789,0.074680226


In [71]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

,par,cov,look,horizon,n,balance,profit
199,0.37,ledoit_wolf,95,10,91,0.102490153,0.316876595
149,0.37,ledoit_wolf,90,10,92,0.088793456,0.289672315
158,0.37,ledoit_wolf,91,9,102,0.092925225,0.289150017
222,0.37,ledoit_wolf,98,3,307,0.110125612,0.288139501
138,0.37,ledoit_wolf,89,9,102,0.089232278,0.285607226
153,0.37,ledoit_wolf,91,4,232,0.107691926,0.284661077
148,0.37,ledoit_wolf,90,9,102,0.087458152,0.284210233
46,0.37,ledoit_wolf,80,7,133,0.097544911,0.284045753
215,0.37,ledoit_wolf,97,6,153,0.101016831,0.281120509
128,0.37,ledoit_wolf,88,9,103,0.095319928,0.280706351


In [67]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

,par,cov,look,horizon,n,balance,profit
199,0.35,ledoit_wolf,95,10,91,0.102404941,0.303427301
158,0.35,ledoit_wolf,91,9,102,0.092925225,0.278199292
222,0.35,ledoit_wolf,98,3,307,0.109691921,0.275417148
149,0.35,ledoit_wolf,90,10,92,0.088793456,0.274179610
138,0.35,ledoit_wolf,89,9,102,0.089232278,0.272283932
46,0.35,ledoit_wolf,80,7,133,0.097476395,0.271506323
148,0.35,ledoit_wolf,90,9,102,0.087458152,0.271206971
153,0.35,ledoit_wolf,91,4,232,0.107211998,0.270897136
128,0.35,ledoit_wolf,88,9,103,0.095319928,0.268818052
143,0.35,ledoit_wolf,90,4,232,0.102069821,0.267836961


In [47]:
portfolio = df_final.to_dict()['W']

buy_data = lookback
for key in portfolio.keys():
    asset_price = df_prices.iloc[buy_data][key]
    balance = balance - portfolio[key] * asset_price
    if balance < 0:
        print('Balance < 0')

for key in portfolio.keys():
    asset_price = df_prices.iloc[sell_data][key]
    balance = balance + portfolio[key] * asset_price
    if balance < 0:
        print('Balance < 0')

NameError: name 'buy_data' is not defined

## Basic

In [121]:
# calculate expected returns and sample covariance amtrix

avg_returns = expected_returns.mean_historical_return(df_prices)
cov_mat = risk_models.sample_cov(df_prices)
cov_mat;

In [11]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 3000)

Expected annual return: 225.7%
Annual volatility: 54.4%
Sharpe Ratio: 4.11


,W
LEND-USDT,660.0
REN-USDT,760.0
ADA-USDT,450.0
KAVA-USDT,1080.0


In [12]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 4000)

Expected annual return: -9.7%
Annual volatility: 26.3%
Sharpe Ratio: -0.45


,W
BTC-USDT,1850.0
BNB-USDT,2140.0
